# Extractive

<div class="alert alert-info">

This tutorial is available as an IPython notebook at [Malaya/example/keyword-extractive](https://github.com/huseinzol05/Malaya/tree/master/example/keyword-extractive).
    
</div>

In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
# !pip3.8 install scikit-learn -U

In [3]:
import malaya

/home/husein/.local/lib/python3.8/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/home/husein/.local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [4]:
# https://www.bharian.com.my/berita/nasional/2020/06/698386/isu-bersatu-tun-m-6-yang-lain-saman-muhyiddin

string = """
Dalam saman itu, plaintif memohon perisytiharan, antaranya mereka adalah ahli BERSATU yang sah, masih lagi memegang jawatan dalam parti (bagi pemegang jawatan) dan layak untuk bertanding pada pemilihan parti.

Mereka memohon perisytiharan bahawa semua surat pemberhentian yang ditandatangani Muhammad Suhaimi bertarikh 28 Mei lalu dan pengesahan melalui mesyuarat Majlis Pimpinan Tertinggi (MPT) parti bertarikh 4 Jun lalu adalah tidak sah dan terbatal.

Plaintif juga memohon perisytiharan bahawa keahlian Muhyiddin, Hamzah dan Muhammad Suhaimi di dalam BERSATU adalah terlucut, berkuat kuasa pada 28 Februari 2020 dan/atau 29 Februari 2020, menurut Fasal 10.2.3 perlembagaan parti.

Yang turut dipohon, perisytiharan bahawa Seksyen 18C Akta Pertubuhan 1966 adalah tidak terpakai untuk menghalang pelupusan pertikaian berkenaan oleh mahkamah.

Perisytiharan lain ialah Fasal 10.2.6 Perlembagaan BERSATU tidak terpakai di atas hal melucutkan/ memberhentikan keahlian semua plaintif.
"""

In [5]:
import re

# minimum cleaning, just simply to remove newlines.
def cleaning(string):
    string = string.replace('\n', ' ')
    string = re.sub('[^A-Za-z\-() ]+', ' ', string).strip()
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

string = cleaning(string)

### Use RAKE algorithm

Original implementation from [https://github.com/aneesha/RAKE](https://github.com/aneesha/RAKE). Malaya added attention mechanism into RAKE algorithm.

```python
def rake(
    string: str,
    model = None,
    vectorizer = None,
    top_k: int = 5,
    atleast: int = 1,
    stopwords = get_stopwords,
    **kwargs
):
    """
    Extract keywords using Rake algorithm.

    Parameters
    ----------
    string: str
    model: Object, optional (default=None)
        Transformer model or any model has `attention` method.
    vectorizer: Object, optional (default=None)
        Prefer `sklearn.feature_extraction.text.CountVectorizer` or,
        `malaya.text.vectorizer.SkipGramCountVectorizer`.
        If None, will generate ngram automatically based on `stopwords`.
    top_k: int, optional (default=5)
        return top-k results.
    ngram: tuple, optional (default=(1,1))
        n-grams size.
    atleast: int, optional (default=1)
        at least count appeared in the string to accept as candidate.
    stopwords: List[str], (default=malaya.texts.function.get_stopwords)
        A callable that returned a List[str], or a List[str], or a Tuple[str]
        For automatic Ngram generator.

    Returns
    -------
    result: Tuple[float, str]
    """
```

#### auto-ngram

This will auto generated N-size ngram for keyword candidates.

In [6]:
malaya.keyword.extractive.rake(string)

[(0.11666666666666665, 'ditandatangani Muhammad Suhaimi bertarikh Mei'),
 (0.08888888888888888, 'mesyuarat Majlis Pimpinan Tertinggi'),
 (0.08888888888888888, 'Seksyen C Akta Pertubuhan'),
 (0.05138888888888888, 'parti bertarikh Jun'),
 (0.04999999999999999, 'keahlian Muhyiddin Hamzah')]

#### auto-gram with Attention

In [7]:
electra = malaya.transformer.huggingface(model = 'mesolitica/electra-base-generator-bahasa-cased')

In [8]:
malaya.keyword.extractive.rake(string, model = electra)

You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


[(0.17997009989539167, 'mesyuarat Majlis Pimpinan Tertinggi'),
 (0.14834545777331348, 'Seksyen C Akta Pertubuhan'),
 (0.12264519202953227, 'ditandatangani Muhammad Suhaimi bertarikh Mei'),
 (0.06489439121974774, 'terlucut berkuat kuasa'),
 (0.057367315322155055, 'menghalang pelupusan pertikaian')]

#### using vectorizer

In [9]:
from malaya.text.vectorizer import SkipGramCountVectorizer

stopwords = malaya.text.function.get_stopwords()
vectorizer = SkipGramCountVectorizer(
    token_pattern = r'[\S]+',
    ngram_range = (1, 3),
    stop_words = stopwords,
    lowercase = False,
    skip = 2
)

In [10]:
malaya.keyword.extractive.rake(string, vectorizer = vectorizer)

[(0.0017052987393271276, 'parti memohon perisytiharan'),
 (0.0017036368782590756, 'memohon perisytiharan BERSATU'),
 (0.0017012023597074357, 'memohon perisytiharan sah'),
 (0.0017012023597074357, 'sah memohon perisytiharan'),
 (0.0016992809994779549, 'perisytiharan BERSATU sah')]

#### fixed-ngram with Attention

In [11]:
malaya.keyword.extractive.rake(string, model = electra, vectorizer = vectorizer)

[(0.0033636377889573383, 'Majlis Pimpinan Tertinggi'),
 (0.0033245625223539293, 'Majlis Pimpinan (MPT)'),
 (0.0032415590393544006, 'mesyuarat Majlis Pimpinan'),
 (0.003145062492212815, 'pengesahan Majlis Pimpinan'),
 (0.003103919348118483, 'Mei Majlis Pimpinan')]

### Use Textrank algorithm

Malaya simply use textrank algorithm.

```python
def textrank(
    string: str,
    model = None,
    vectorizer = None,
    top_k: int = 5,
    atleast: int = 1,
    stopwords = get_stopwords,
    **kwargs
):
    """
    Extract keywords using Textrank algorithm.

    Parameters
    ----------
    string: str
    model: Object, optional (default='None')
        model has `fit_transform` or `vectorize` method.
    vectorizer: Object, optional (default=None)
        Prefer `sklearn.feature_extraction.text.CountVectorizer` or, 
        `malaya.text.vectorizer.SkipGramCountVectorizer`.
        If None, will generate ngram automatically based on `stopwords`.
    top_k: int, optional (default=5)
        return top-k results.
    atleast: int, optional (default=1)
        at least count appeared in the string to accept as candidate.
    stopwords: List[str], (default=malaya.texts.function.get_stopwords)
        A callable that returned a List[str], or a List[str], or a Tuple[str]

    Returns
    -------
    result: Tuple[float, str]
    """
```

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

#### auto-ngram with TFIDF

This will auto generated N-size ngram for keyword candidates.

In [13]:
malaya.keyword.extractive.textrank(string, model = tfidf)

[(0.00015733542794311347, 'plaintif memohon perisytiharan'),
 (0.00012558967338659324, 'Fasal perlembagaan parti'),
 (0.00011514136972371928, 'Fasal Perlembagaan BERSATU'),
 (0.00011505529351381784, 'parti'),
 (0.00010763518993348075, 'memohon perisytiharan')]

#### auto-ngram with Attention

This will auto generated N-size ngram for keyword candidates.

In [14]:
malaya.transformer.available_huggingface

{'mesolitica/roberta-base-bahasa-cased': {'Size (MB)': 443},
 'mesolitica/roberta-tiny-bahasa-cased': {'Size (MB)': 66.1},
 'mesolitica/bert-base-standard-bahasa-cased': {'Size (MB)': 443},
 'mesolitica/bert-tiny-standard-bahasa-cased': {'Size (MB)': 66.1},
 'mesolitica/roberta-base-standard-bahasa-cased': {'Size (MB)': 443},
 'mesolitica/roberta-tiny-standard-bahasa-cased': {'Size (MB)': 66.1},
 'mesolitica/electra-base-generator-bahasa-cased': {'Size (MB)': 140},
 'mesolitica/electra-small-generator-bahasa-cased': {'Size (MB)': 19.3}}

In [15]:
electra = malaya.transformer.huggingface(model = 'mesolitica/electra-small-generator-bahasa-cased')
roberta = malaya.transformer.huggingface(model = 'mesolitica/roberta-tiny-bahasa-cased')

In [16]:
malaya.keyword.extractive.textrank(string, model = electra)

You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


[(6.318265638774778e-05, 'dipohon perisytiharan'),
 (6.316746347908478e-05, 'pemegang jawatan'),
 (6.316118494856124e-05, 'parti bertarikh Jun'),
 (6.316104081180683e-05, 'Februari'),
 (6.315819096453449e-05, 'plaintif')]

In [17]:
malaya.keyword.extractive.textrank(string, model = roberta)

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


[(6.592447573341062e-05, 'parti'),
 (6.584500366507439e-05, 'keahlian Muhyiddin Hamzah'),
 (6.48854995406703e-05, 'dipohon perisytiharan'),
 (6.4585235386134e-05, 'surat pemberhentian'),
 (6.436106705986197e-05, 'parti bertarikh Jun')]

#### fixed-ngram with Attention

In [18]:
stopwords = malaya.text.function.get_stopwords()
vectorizer = SkipGramCountVectorizer(
    token_pattern = r'[\S]+',
    ngram_range = (1, 3),
    stop_words = stopwords,
    lowercase = False,
    skip = 2
)

In [19]:
malaya.keyword.extractive.textrank(string, model = electra, vectorizer = vectorizer)

[(5.6521309567750785e-09, 'plaintif perisytiharan'),
 (5.6520371534856045e-09, 'perisytiharan ahli sah'),
 (5.6519577794113005e-09, 'Plaintif perisytiharan keahlian'),
 (5.651893084611097e-09, 'Perisytiharan'),
 (5.651664613022757e-09, 'plaintif memohon perisytiharan')]

In [20]:
malaya.keyword.extractive.textrank(string, model = roberta, vectorizer = vectorizer)

[(5.923583228770409e-09, 'keahlian Muhyiddin Muhammad'),
 (5.9165313140370156e-09, 'parti bertarikh'),
 (5.913081883473326e-09, 'kuasa Fasal'),
 (5.902597482992337e-09, 'C Akta menghalang'),
 (5.90093003514351e-09, 'keahlian Muhyiddin')]

### Use Attention mechanism

Use attention mechanism from transformer model to get important keywords.

```python
def attention(
    string: str,
    model,
    vectorizer = None,
    top_k: int = 5,
    atleast: int = 1,
    stopwords = get_stopwords,
    **kwargs
):
    """
    Extract keywords using Attention mechanism.

    Parameters
    ----------
    string: str
    model: Object
        Transformer model or any model has `attention` method.
    vectorizer: Object, optional (default=None)
        Prefer `sklearn.feature_extraction.text.CountVectorizer` or, 
        `malaya.text.vectorizer.SkipGramCountVectorizer`.
        If None, will generate ngram automatically based on `stopwords`.
    top_k: int, optional (default=5)
        return top-k results.
    atleast: int, optional (default=1)
        at least count appeared in the string to accept as candidate.
    stopwords: List[str], (default=malaya.texts.function.get_stopwords)
        A callable that returned a List[str], or a List[str], or a Tuple[str]

    Returns
    -------
    result: Tuple[float, str]
    """
```

#### auto-ngram

This will auto generated N-size ngram for keyword candidates.

In [21]:
malaya.keyword.extractive.attention(string, model = electra)

[(0.7273892781585295, 'menghalang pelupusan pertikaian'),
 (0.037768079524707135, 'plaintif memohon perisytiharan'),
 (0.03168171774504689, 'dipohon perisytiharan'),
 (0.03101700203360666, 'memohon perisytiharan'),
 (0.02176717370447907, 'ditandatangani Muhammad Suhaimi bertarikh Mei')]

In [22]:
malaya.keyword.extractive.attention(string, model = roberta)

[(0.07387573112018977, 'plaintif memohon perisytiharan'),
 (0.06143066429484969, 'Fasal perlembagaan parti'),
 (0.05755474756860026, 'ditandatangani Muhammad Suhaimi bertarikh Mei'),
 (0.05666392261960079, 'Fasal Perlembagaan BERSATU'),
 (0.05564947309701604, 'memohon perisytiharan')]

#### fixed-ngram

In [23]:
malaya.keyword.extractive.attention(string, model = electra, vectorizer = vectorizer)

[(0.029306968080037014, 'pertikaian Perisytiharan Fasal'),
 (0.029205818109817004, 'pertikaian mahkamah Fasal'),
 (0.02919627468587057, 'pertikaian Fasal Perlembagaan'),
 (0.02918733579124283, 'pelupusan pertikaian Fasal'),
 (0.029172178345266454, 'pertikaian Fasal')]

In [24]:
malaya.keyword.extractive.attention(string, model = roberta, vectorizer = vectorizer)

[(0.003424167402512117, 'parti memohon perisytiharan'),
 (0.0032962148932236144, 'memohon perisytiharan BERSATU'),
 (0.0031886482418839175, 'plaintif perisytiharan BERSATU'),
 (0.003181526276520311, 'BERSATU sah parti'),
 (0.0031634494027450396, 'perisytiharan BERSATU sah')]

### Use similarity mechanism

```python
def similarity(
    string: str,
    model,
    vectorizer = None,
    top_k: int = 5,
    atleast: int = 1,
    stopwords = get_stopwords,
    **kwargs,
):
    """
    Extract keywords using Sentence embedding VS keyword embedding similarity.

    Parameters
    ----------
    string: str
    model: Object
        Transformer model or any model has `vectorize` method.
    vectorizer: Object, optional (default=None)
        Prefer `sklearn.feature_extraction.text.CountVectorizer` or, 
        `malaya.text.vectorizer.SkipGramCountVectorizer`.
        If None, will generate ngram automatically based on `stopwords`.
    top_k: int, optional (default=5)
        return top-k results.
    atleast: int, optional (default=1)
        at least count appeared in the string to accept as candidate.
    stopwords: List[str], (default=malaya.texts.function.get_stopwords)
        A callable that returned a List[str], or a List[str], or a Tuple[str]

    Returns
    -------
    result: Tuple[float, str]
    """
```

In [25]:
malaya.keyword.extractive.similarity(string, model = roberta)

[(0.8739699, 'plaintif memohon perisytiharan'),
 (0.8719046, 'keahlian Muhyiddin Hamzah'),
 (0.8637232, 'mahkamah Perisytiharan'),
 (0.86043775, 'dipohon perisytiharan'),
 (0.8529164, 'memohon perisytiharan')]

In [26]:
malaya.keyword.extractive.similarity(string, model = electra)

[(0.9982966, 'keahlian Muhyiddin Hamzah'),
 (0.99825895, 'mesyuarat Majlis Pimpinan Tertinggi'),
 (0.9981606, 'Fasal perlembagaan parti'),
 (0.9981444, 'Fasal Perlembagaan BERSATU'),
 (0.9979403, 'ditandatangani Muhammad Suhaimi bertarikh Mei')]